# Day 7: Amplification Circuit

## Part One

<pre>
Based on the navigational maps, you're going to need to send more power to your ship's thrusters to reach Santa in time. To do this, you'll need to configure a series of amplifiers already installed on the ship.

There are five amplifiers connected in series; each one receives an input signal and produces an output signal. They are connected such that the first amplifier's output leads to the second amplifier's input, the second amplifier's output leads to the third amplifier's input, and so on. The first amplifier's input value is 0, and the last amplifier's output leads to your ship's thrusters.
</pre>

<pre>
    O-------O  O-------O  O-------O  O-------O  O-------O
0 ->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-> (to thrusters)
    O-------O  O-------O  O-------O  O-------O  O-------O
</pre>

<pre>
The Elves have sent you some Amplifier Controller Software (your puzzle input), a program that should run on your existing Intcode computer. Each amplifier will need to run a copy of the program.

When a copy of the program starts running on an amplifier, it will first use an input instruction to ask the amplifier for its current phase setting (an integer from 0 to 4). Each phase setting is used exactly once, but the Elves can't remember which amplifier needs which phase setting.

The program will then call another input instruction to get the amplifier's input signal, compute the correct output signal, and supply it back to the amplifier with an output instruction. (If the amplifier has not yet received an input signal, it waits until one arrives.)

Your job is to find the largest output signal that can be sent to the thrusters by trying every possible combination of phase settings on the amplifiers. Make sure that memory is not shared or reused between copies of the program.

For example, suppose you want to try the phase setting sequence 3,1,2,4,0, which would mean setting amplifier A to phase setting 3, amplifier B to setting 1, C to 2, D to 4, and E to 0. Then, you could determine the output signal that gets sent from amplifier E to the thrusters with the following steps:

Start the copy of the amplifier controller software that will run on amplifier A. At its first input instruction, provide it the amplifier's phase setting, 3. At its second input instruction, provide it the input signal, 0. After some calculations, it will use an output instruction to indicate the amplifier's output signal.
Start the software for amplifier B. Provide it the phase setting (1) and then whatever output signal was produced from amplifier A. It will then produce a new output signal destined for amplifier C.
Start the software for amplifier C, provide the phase setting (2) and the value from amplifier B, then collect its output signal.
Run amplifier D's software, provide the phase setting (4) and input value, and collect its output signal.
Run amplifier E's software, provide the phase setting (0) and input value, and collect its output signal.
The final output signal from amplifier E would be sent to the thrusters. However, this phase setting sequence may not have been the best one; another sequence might have sent a higher signal to the thrusters.

Here are some example programs:

Max thruster signal 43210 (from phase setting sequence 4,3,2,1,0):

3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0
Max thruster signal 54321 (from phase setting sequence 0,1,2,3,4):

3,23,3,24,1002,24,10,24,1002,23,-1,23,
101,5,23,23,1,24,23,23,4,23,99,0,0
Max thruster signal 65210 (from phase setting sequence 1,0,4,3,2):

3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,
1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0

</pre>

Try every combination of phase settings on the amplifiers. **What is the highest signal that can be sent to the thrusters?**

To begin, get your puzzle input.


In [14]:
# Import the same code from Day 5 to modify

def parse_long_direction(direction):
    shortopcode = str(direction).zfill(5)
    param_1_mode, param_2_mode, param_3_mode, opcode = int(shortopcode[2]), int(shortopcode[1]), int(shortopcode[0]), int(shortopcode[-2:])
    direction_list = [opcode, param_1_mode, param_2_mode, param_3_mode]
    return direction_list

In [135]:
def diagnosis_code(program, phase, output, step=0):
    opcode_index = 0
    first_time = 0
    while True:
        opcode = program[opcode_index]
        direction = parse_long_direction(opcode)[0]
        elements = []
        if direction in {1, 2, 7, 8}:
            elements_length = 3
        elif direction in {3, 4}:
            elements_length = 1
        elif direction in {5,6}:
            elements_length = 2
        for i in range(1,elements_length+1):
            if parse_long_direction(opcode)[i] == 0:
                elements.append(program[opcode_index + i])
            else:
                elements.append(opcode_index + i)
        if direction == 99:
                #phase = phases_1[step]
                break   
        elif direction == 1:
            program[elements[2]] = program[elements[0]] + program[elements[1]]
            opcode_index +=4
        elif direction == 2:
            program[elements[2]] = program[elements[0]] * program[elements[1]]
            opcode_index +=4
        elif direction == 3:
            if first_time == 0:
                first_time +=1
                program[elements[0]] = phase
            else:
                program[elements[0]] = output
            opcode_index +=2
        elif direction == 4:
            new_input = program[elements[0]]
            opcode_index +=2
        elif direction == 5:
            if program[elements[0]] != 0:
                opcode_index = program[elements[1]]
            else:
                opcode_index +=3
        elif direction == 6:
            if program[elements[0]] == 0:
                opcode_index = program[elements[1]]
            else:
                opcode_index +=3
        elif direction == 7:
            if program[elements[0]] < program[elements[1]]:
                program[elements[2]] = 1
                opcode_index +=4
            else:
                program[elements[2]] = 0
                opcode_index +=4
        elif direction == 8:
            if program[elements[0]] == program[elements[1]]:
                program[elements[2]] = 1
                opcode_index +=4
            else:
                program[elements[2]] = 0
                opcode_index +=4
    #program = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
    #diagnosis_code(program, phase, output, step)
    #list_of_inputs.append(new_input)
    return new_input

In [136]:
def pass_through_amplifiers(program, phase, step=0):
    outputs_list = [1]
    for step in range(5):
        output = outputs_list[step]
        phase = phases_1[step]
        outputs_list.append(diagnosis_code(program, phase, output, step))
    print(outputs_list[-1])

In [137]:
test_intcode_1 = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
# test_intcode_2 = [3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0]

In [138]:
phases_1 = [4,3,2,1,0]
#phases_2 = [0,1,2,3,4]
pass_through_amplifiers(test_intcode_1, 4, 0)

143210
